In [1]:
import pandas as pd
import urllib.parse
from selenium import webdriver
from bs4 import BeautifulSoup

# CSV 파일에서 데이터 읽어오기
data = pd.read_csv('Nowon.csv', encoding='cp949')

# 음식과 숙박을 제외한 행만 필터링
filtered_data = data[(data['KTO카테고리대분류명'] != '음식') & (data['KTO카테고리대분류명'] != '숙박')]

# 관광지명 추출
tourist_spots = filtered_data['관광지명'].tolist()

# 중복 관광지명 제거
tourist_spots = list(set(tourist_spots))

# Chrome WebDriver 초기화
options = webdriver.ChromeOptions()
options.add_argument('headless')  # 브라우저 창이 나타나지 않도록 설정
driver = webdriver.Chrome(options=options)

# 결과를 저장할 데이터 프레임 생성
result_df = pd.DataFrame(columns=['관광지명', '음식점명', 'URL'])

# 관광지명에 대해 반복
for spot in tourist_spots:
    # Google 지도 페이지로 이동
    url = f"https://www.google.com/maps/search/{urllib.parse.quote(spot)}+%EB%A7%9B%EC%A7%91/data=!3m1!4b1?hl=ko&entry=ttu"
    driver.get(url)
    
    # 페이지의 HTML 가져오기
    page_content = driver.page_source
    
    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(page_content, 'html.parser')
    
    # 클래스가 'hfpxzc'인 모든 요소(맛집) 찾기
    elements = soup.find_all('a', class_='hfpxzc')
    
    # 관광지별 음식점 목록을 저장할 리스트 초기화
    restaurants = []
    
    # 결과를 데이터프레임에 추가
    for element in elements:
        aria_label = element['aria-label']
        href = element['href']
        
        # 중복 음식점 제거
        if aria_label not in restaurants:
            restaurants.append(aria_label)
            # 결과 데이터프레임에 추가
            result_df = pd.concat([result_df, pd.DataFrame({'관광지명': [spot], '음식점명': [aria_label], 'URL': [href]})], ignore_index=True)
    
    # 관광지별로 음식점명이 안 나오는 경우 해당 관광지 삭제
    if len(restaurants) == 0:
        result_df = result_df[result_df['관광지명'] != spot]
        
# 음식점명과 관광지명이 겹치는 경우 해당 행 삭제
result_df = result_df[result_df['음식점명'] != result_df['관광지명']]

# 크롤링이 끝나면 WebDriver 종료
driver.quit()

# 결과 데이터프레임 출력
print(result_df)

             관광지명          음식점명  \
0          피노파밀리아      피제리아 일피노   
1          피노파밀리아          월드피노   
3    광운대학교 아이스링크장           이층집   
4        서울생활사박물관  서울역사박물관 구내식당   
5        서울생활사박물관        거울못 식당   
..            ...           ...   
188           초안산        민들레 초밥   
189           초안산      소문난규리네맛집   
190           초안산    하다식당 월계직영점   
191           초안산           고빠루   
192           초안산          한끼뚝딱   

                                                   URL  
0    https://www.google.com/maps/place/%ED%94%BC%EC...  
1    https://www.google.com/maps/place/%EC%9B%94%EB...  
3    https://www.google.com/maps/place/%EC%9D%B4%EC...  
4    https://www.google.com/maps/place/%EC%84%9C%EC...  
5    https://www.google.com/maps/place/%EA%B1%B0%EC...  
..                                                 ...  
188  https://www.google.com/maps/place/%EB%AF%BC%EB...  
189  https://www.google.com/maps/place/%EC%86%8C%EB...  
190  https://www.google.com/maps/place/%ED%95%98%EB...  
191  http

In [2]:
# 결과 데이터프레임을 CSV 파일로 저장
result_df.to_csv('Nowon_eat.csv', index=False, encoding='utf-8-sig')